In [5]:
import pandas as pd
import pathlib
import ast
import numpy as np
import math
from collections import defaultdict
from collections import Counter
import re

BASE_DIR = pathlib.Path.cwd().parent.parent.resolve()
BASE_DIR

WindowsPath('C:/Users/aleja/Documents/GitHub/SilkNews')

In [6]:
df = pd.read_csv(f"{BASE_DIR}/src/data/extraccion_caracteristicas/extraccion_caracteristicas.csv")
df.head()

,text,label,texto_limpio,sin_stopwords,frecuencia<3
0,en una rueda de prensa las autoridades dijeron...,True,en una rued de prens las autor dijeron que los...,"{'rued': 19, 'prens': 13, 'autor': 0, 'familia...","{'rued': 19, 'prens': 13, 'familiar': 6, 'robi..."
1,una operacion en la que las autoridades irland...,True,una oper en la que las autor irlandes decomis ...,"{'oper': 17, 'autor': 0, 'irlandes': 13, 'deco...","{'oper': 17, 'irlandes': 13, 'decomis': 4, 'to..."
2,el nuevo informe se basa en mas de entrevistas...,True,el nuev inform se bas en mas de entrev con per...,"{'nuev': 13, 'inform': 9, 'bas': 1, 'entrev': ...","{'nuev': 13, 'inform': 9, 'entrev': 5, 'person..."
3,las pruebas hechas a varias de las principales...,True,las prueb hech a vari de las principal marc ar...,"{'prueb': 9, 'hech': 2, 'vari': 13, 'principal...","{'prueb': 9, 'vari': 13, 'principal': 7, 'marc..."
4,michelle y john wylie estuvieron buscando dura...,True,michell y john wyli estuv busc durant anos a u...,"{'michell': 15, 'john': 13, 'wyli': 21, 'estuv...","{'michell': 15, 'john': 13, 'wyli': 21, 'estuv..."


## Frecuencia Absoluta 

In [12]:
def calcular_frecuencia_palabras(texto):
    """Calcula la frecuencia de palabras en un texto"""
    if pd.isna(texto):
        return {}
    
    palabras = re.findall(r'\b[a-záéíóúñü]+\b', texto.lower())

    frecuencia = Counter(palabras)
    return dict(frecuencia)

df['frecuencia_palabras'] = df['texto_limpio'].apply(calcular_frecuencia_palabras)
for i in range(min(3, len(df))):
    print(f"\nTexto {i+1}:")
    print(f"Frecuencias: {dict(list(df.iloc[i]['frecuencia_palabras'].items())[:10])}")


Texto 1:
Frecuencias: {'en': 1, 'una': 1, 'rued': 1, 'de': 3, 'prens': 1, 'las': 2, 'autor': 1, 'dijeron': 1, 'que': 1, 'los': 1}

Texto 2:
Frecuencias: {'una': 1, 'oper': 1, 'en': 1, 'la': 2, 'que': 2, 'las': 1, 'autor': 1, 'irlandes': 1, 'decomis': 1, 'mas': 1}

Texto 3:
Frecuencias: {'el': 1, 'nuev': 1, 'inform': 1, 'se': 1, 'bas': 1, 'en': 2, 'mas': 1, 'de': 2, 'entrev': 1, 'con': 1}


## Esquema TF-IDF

In [8]:
def convertir_a_dict(cell_value):
    if isinstance(cell_value, str):
        try:
            return ast.literal_eval(cell_value)
        except:
            return {}
    return cell_value if isinstance(cell_value, dict) else {}

documentos_freq = []
for idx, freq_str in df['frecuencia<3'].items():
    doc_dict = convertir_a_dict(freq_str)
    documentos_freq.append(doc_dict)

vocabulario = set()
for doc in documentos_freq:
    vocabulario.update(doc.keys())

vocabulario = sorted(list(vocabulario))
print(f"Tamaño del vocabulario: {len(vocabulario)}")

df_terminos = defaultdict(int)
for doc in documentos_freq:
    for termino in doc.keys():
        df_terminos[termino] += 1

Tamaño del vocabulario: 258800


In [9]:
def calcular_tf_idf(documentos_freq, df_terminos, vocabulario):
    N = len(documentos_freq)
    tfidf_matrix = []
    
    for i, doc in enumerate(documentos_freq):
        total_words_in_doc = sum(doc.values())
        doc_tfidf = {}
        
        for termino in doc:
            tf = doc[termino] / total_words_in_doc
            
            idf = math.log(N / df_terminos[termino])
            
            tfidf = tf * idf
            doc_tfidf[termino] = tfidf
        
        tfidf_matrix.append(doc_tfidf)
    
    return tfidf_matrix

tfidf_matrix = calcular_tf_idf(documentos_freq, df_terminos, vocabulario)

print(f"Matriz TF-IDF calculada con {len(tfidf_matrix)} documentos")

Matriz TF-IDF calculada con 5568 documentos


In [10]:
terminos_tfidf_promedio = defaultdict(list)
for doc in tfidf_matrix:
    for termino, tfidf_val in doc.items():
        terminos_tfidf_promedio[termino].append(tfidf_val)

terminos_stats = {}
for termino, valores in terminos_tfidf_promedio.items():
    terminos_stats[termino] = {
        'promedio': np.mean(valores),
        'max': np.max(valores),
        'min': np.min(valores),
        'std': np.std(valores),
        'docs_count': len(valores)
    }

terminos_ordenados = sorted(terminos_stats.items(), key=lambda x: x[1]['promedio'], reverse=True)

print("terminos con mayor TF-IDF promedio:")
print("-" * 60)
for i, (termino, stats) in enumerate(terminos_ordenados[:5]):
    print(f"{i+1:2d}. {termino:<20} | Promedio: {stats['promedio']:.6f} | Docs: {stats['docs_count']:4d}")

print(f"terminos con mayor TF-IDF máximo:")
print("-" * 60)
terminos_max = sorted(terminos_stats.items(), key=lambda x: x[1]['max'], reverse=True)
for i, (termino, stats) in enumerate(terminos_max[:5]):
    print(f"{i+1:2d}. {termino:<20} | Máximo: {stats['max']:.6f} | Docs: {stats['docs_count']:4d}")

terminos con mayor TF-IDF promedio:
------------------------------------------------------------
 1. xiaomi vend          | Promedio: 2.090858 | Docs:    1
 2. sac call             | Promedio: 2.090858 | Docs:    1
 3. muert johnny         | Promedio: 2.090858 | Docs:    1
 4. veteran espan        | Promedio: 1.922823 | Docs:    2
 5. vigil espectacul     | Promedio: 1.922823 | Docs:    2
terminos con mayor TF-IDF máximo:
------------------------------------------------------------
 1. vien trabaj          | Máximo: 2.338451 | Docs:    5
 2. xiaomi vend          | Máximo: 2.090858 | Docs:    1
 3. sac call             | Máximo: 2.090858 | Docs:    1
 4. muert johnny         | Máximo: 2.090858 | Docs:    1
 5. veteran espan        | Máximo: 1.922823 | Docs:    2
 1. vien trabaj          | Máximo: 2.338451 | Docs:    5
 2. xiaomi vend          | Máximo: 2.090858 | Docs:    1
 3. sac call             | Máximo: 2.090858 | Docs:    1
 4. muert johnny         | Máximo: 2.090858 | Docs:    1


In [11]:
tfidf_data = []
for i, doc_tfidf in enumerate(tfidf_matrix):
    for termino, tfidf_val in doc_tfidf.items():
        tfidf_data.append({
            'termino': termino,
            'tfidf': tfidf_val,
        })

df_tfidf = pd.DataFrame(tfidf_data)

print(f"Número total de entradas término-documento: {len(df_tfidf)}")
print(f"TF-IDF promedio: {df_tfidf['tfidf'].mean():.6f}")
print(f"TF-IDF máximo: {df_tfidf['tfidf'].max():.6f}")
print(f"TF-IDF mínimo: {df_tfidf['tfidf'].min():.6f}")

df_tfidf

Número total de entradas término-documento: 772685
TF-IDF promedio: 0.042972
TF-IDF máximo: 2.338451
TF-IDF mínimo: 0.000004


,termino,tfidf
0,rued,0.392941
1,prens,0.224112
2,familiar,0.120216
3,robinson,0.708316
4,hic,0.171782
...,...,...
772680,cuatr pat,0.006659
772681,pat ano,0.021514
772682,ano apunt,0.001537
772683,apunt haci,0.001542
